Oct 12th 2025

## Objective:
Use previously pre-processed data and fit a suite of classification models in a modular pipeline. The models chosen are logistic regression, Decision Tree and Random Forest. Optimise all 3 models for hyperparamters and evaluate performance using ROC-AUC curve and accuracy. Finally write the optimized model objects in artifacts